In [ ]:
from gs_quant.session import GsSession, Environment
client_id = None # Supply your application id
client_secret = None # Supply your client secret
scopes = ('run_analytics',)

GsSession.use(Environment.PROD, client_id, client_secret, scopes)

In [ ]:
from gs_quant.instrument import IRSwap
from gs_quant.risk import IRFwdRate, CarryScenario
from gs_quant.markets.portfolio import Portfolio
from datetime import datetime
import matplotlib.pylab as plt
import pandas as pd

In [ ]:
ccy = 'EUR'

# construct a series of 6m FRAs going out 20y or so
fras = Portfolio([IRSwap('Pay', '{}m'.format(i), ccy, effective_date='{}m'.format(i-6), fixed_rate_frequency='6m', floating_rate_frequency='6m') for i in range(6, 123, 6)])
fras.resolve()
results = fras.calc(IRFwdRate)

# get the fwd rates for these fras under the base sceneraio (no shift in time)
base = {}
for i, res in enumerate(results):
    base[datetime.strptime(fras[i].termination_date, '%Y-%m-%d')] = res

base_series = pd.Series(base, name='base')

In [ ]:
# calculate the fwd rates with a shift forward of 132 business days - about 6m.  This shift keeps spot rates constant.  
# So 5y rate today will be 5y rate under the scenario of pricing 6m in the future.
with CarryScenario(time_shift=132, roll_to_fwds=False):
    fras = Portfolio([IRSwap('Pay', '{}m'.format(i), ccy, effective_date='{}m'.format(i-6), fixed_rate_frequency='6m', floating_rate_frequency='6m') for i in range(6, 123, 6)])
    fras.resolve()
    results = fras.calc(IRFwdRate)

    roll_spot = {}
    for i, res in enumerate(results):
        roll_spot[datetime.strptime(fras[i].termination_date, '%Y-%m-%d')] = res

roll_spot_series = pd.Series(roll_spot, name='roll to spot')


In [ ]:
# calculate the fwd rates with a shift forward of 132 business days - about 6m.  This shift keeps fwd rates constant.  
# So 5.5y rate today will be 5y rate under the scenario of pricing 6m in the future.

with CarryScenario(time_shift=132, roll_to_fwds=True):
    fras = Portfolio([IRSwap('Pay', '{}m'.format(i), ccy, effective_date='{}m'.format(i-6), fixed_rate_frequency='6m', floating_rate_frequency='6m') for i in range(6, 123, 6)])
    fras.resolve()
    results = fras.calc(IRFwdRate)

    roll_fwd = {}
    for i, res in enumerate(results):
        roll_fwd[datetime.strptime(fras[i].termination_date, '%Y-%m-%d')] = res

roll_fwd_series = pd.Series(roll_fwd, name='roll to fwd')


In [ ]:
# show the curves, the base in blue, the roll to fwd in green and the roll to spot in orange.
# note blue and green curves are not exactly on top of each other as we aren't using the curve instruments themselves
# but instead using FRAs to show a smooth curve.
base_series.plot(figsize=(20,10))
roll_spot_series.plot()
roll_fwd_series.plot()
plt.legend()

